In [162]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
import keras_tuner

from tensorboard.plugins.hparams import api as hp

In [163]:
DATA_VERSION = "v2"

merged_data_path = f'../data/{DATA_VERSION}/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

In [164]:
# K-MEANS

# Prepare the list of unique genres
unique_genres = list(set([genre for genres in data['genres'] for genre in genres]) | set(
    [genre for genres in data['favourite_genres'] for genre in genres]))

# Convert the genres to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(unique_genres)

# Apply K-means clustering
num_clusters = 100  # Adjust this value according to your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
kmeans.fit(vectorized)
labels = kmeans.labels_

# Group the genres based on the cluster labels
clustered_genres = {}
for i, label in enumerate(labels):
    if label not in clustered_genres:
        clustered_genres[label] = []
    clustered_genres[label].append(unique_genres[i])

# Map genres to their cluster label
genre_to_cluster = {genre: label for label, genres in clustered_genres.items() for genre in genres}


# Define the function for mapping genres to simpler forms
def map_genre(genre):
    cluster_label = genre_to_cluster[genre]
    representative_genre = clustered_genres[cluster_label][
        0]  # Use the first genre in the cluster as the representative
    return representative_genre


# Save the genre_to_cluster and clustered_genres dictionaries to use for new data prediction
with open('../microservice/models/model2_genre_to_cluster.pickle', 'wb') as f:
    pickle.dump(genre_to_cluster, f)

with open('../microservice/models/model2_clustered_genres.pickle', 'wb') as f:
    pickle.dump(clustered_genres, f)

# Apply the mapping function to both 'genres' and 'favourite_genres' columns
data['genres'] = data['genres'].apply(lambda x: [map_genre(genre) for genre in x])
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: [map_genre(genre) for genre in x])

# Delete not unique genres
data['genres'] = data['genres'].apply(lambda x: list(set(x)))
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: list(set(x)))


In [165]:
# count unique genres
unique_genres = set()
for genres in data['genres']:
    unique_genres.update(genres)
for genres in data['favourite_genres']:
    unique_genres.update(genres)
print(len(unique_genres))

# how many tracks are per genre
genre_count = {}
for genres in data['genres']:
    for genre in genres:
        if genre not in genre_count:
            genre_count[genre] = 0
        genre_count[genre] += 1

genre_count = sorted(genre_count.items(), key=lambda x: x[1], reverse=True)
genre_count

100


[('harlem renaissance', 6923),
 ('polish alternative', 3707),
 ('slovak pop', 2860),
 ('australian indie folk', 1571),
 ('hard bop', 1140),
 ('art punk', 1126),
 ('country road', 991),
 ('new wave', 916),
 ('roots rock', 793),
 ('drone metal', 657),
 ('russian dance', 619),
 ('uk hip hop', 554),
 ('taiwan singer-songwriter', 544),
 ('gangster rap', 499),
 ('british dance band', 470),
 ('psychedelic soul', 447),
 ('uk doom metal', 445),
 ('progressive bluegrass', 408),
 ('latin', 373),
 ('trap latino', 373),
 ('neon pop punk', 329),
 ('early modern classical', 321),
 ('funk', 319),
 ('jazz organ', 311),
 ('english indie rock', 242),
 ('canadian folk', 228),
 ('brazilian emo', 202),
 ('acid house', 197),
 ('piano rock', 173),
 ('chicago drill', 165),
 ('spanish rockabilly', 151),
 ('southern rock', 142),
 ('us power metal', 139),
 ('melodic thrash', 131),
 ('canadian pop punk', 112),
 ('modern reggae', 110),
 ('mexican son', 108),
 ('swing italiano', 108),
 ('german literature', 105),
 (

In [166]:
# balance set
skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

skipped_data (3621, 3)
not_skipped_data (6260, 3)


In [167]:
# Combine genres and favourite_genres
mlb = MultiLabelBinarizer()

data['combined_genres'] = data.apply(lambda x: x['genres'] + x['favourite_genres'], axis=1)
mlb.fit(data['combined_genres'])

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])

# save mlb to file
with open('../microservice/models/model2_mlb.pickle', 'wb') as f:
    pickle.dump(mlb, f)

In [168]:
X = np.concatenate([encoded_genres, encoded_favourite_genres], axis=1)

# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_train == 0))
print("Y_train skipped", np.count_nonzero(y_train == 1))
print("Y_train skipped %", np.count_nonzero(y_train == 1) / len(y_train))

Y_train not skipped 2895
Y_train skipped 2898
Y_train skipped % 0.5002589331952356


In [169]:
mode = "train"
# mode = "tune"

# NOTATKI Z TRAINU
# 2 layer - 111,41  neuronow 0.003 learning rate (20 epok, 0.670 acc) dropout (0.5)

# NOTATKI Z TUNINGU


In [170]:
def build_model(hp):
    if mode == "train":
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1])))
        model.add(Dense(units=111, activation="relu"))
        model.add(Dropout(0.5))
        model.add(Dense(units=41, activation="relu"))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation="sigmoid"))

        model.compile(
            optimizer=Adam(learning_rate=0.003),
            loss='binary_crossentropy', metrics=['accuracy'])
    else:
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1])))

        for i in range(hp.Int("num_layers", min_value=1, max_value=3)):
            model.add(Dense(units=hp.Int(f"units_{i}", min_value=1, max_value=1000, step=5), activation="relu"))
            if hp.Boolean("dropout"):
                model.add(Dropout(0.5))

        model.add(Dense(1, activation="sigmoid"))

        model.compile(optimizer=Adam(learning_rate=hp.Float("learning_rate", min_value=0.001, max_value=0.01, sampling="log")),loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [171]:
if mode == "train":
    model = build_model(None)
    model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=20,
        batch_size=32,
        verbose=2
    )
else:
    tuner = keras_tuner.RandomSearch(
        hypermodel=build_model,
        objective="val_accuracy",
        max_trials=100,
        executions_per_trial=1,
        overwrite=True,
        directory="tuner",
        project_name="IUM",
    )

    stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir="logs")

    tuner.search_space_summary()

    tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[stop_early, tensorboard_callback])

    tuner.results_summary()

Epoch 1/20
182/182 - 3s - loss: 0.6844 - accuracy: 0.5574 - val_loss: 0.6626 - val_accuracy: 0.5935 - 3s/epoch - 16ms/step
Epoch 2/20
182/182 - 2s - loss: 0.6633 - accuracy: 0.5869 - val_loss: 0.6492 - val_accuracy: 0.5977 - 2s/epoch - 8ms/step
Epoch 3/20
182/182 - 1s - loss: 0.6515 - accuracy: 0.6107 - val_loss: 0.6370 - val_accuracy: 0.6156 - 1s/epoch - 7ms/step
Epoch 4/20
182/182 - 1s - loss: 0.6445 - accuracy: 0.6059 - val_loss: 0.6353 - val_accuracy: 0.6149 - 1s/epoch - 7ms/step
Epoch 5/20
182/182 - 2s - loss: 0.6382 - accuracy: 0.6268 - val_loss: 0.6379 - val_accuracy: 0.6308 - 2s/epoch - 11ms/step
Epoch 6/20
182/182 - 2s - loss: 0.6305 - accuracy: 0.6389 - val_loss: 0.6394 - val_accuracy: 0.6356 - 2s/epoch - 9ms/step
Epoch 7/20
182/182 - 1s - loss: 0.6239 - accuracy: 0.6437 - val_loss: 0.6364 - val_accuracy: 0.6391 - 1s/epoch - 7ms/step
Epoch 8/20
182/182 - 1s - loss: 0.6157 - accuracy: 0.6456 - val_loss: 0.6335 - val_accuracy: 0.6494 - 1s/epoch - 8ms/step
Epoch 9/20
182/182 - 2

In [172]:
if mode == "train":
    # save model to file
    model.save('../microservice/models/model2.h5')

    print("TEST")
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y_test, y_pred_classes))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
    print("Classification report:\n", classification_report(y_test, y_pred_classes))

    print("TRAIN")
    y_pred = model.predict(X_train)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y_train, y_pred_classes))
    print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
    print("Classification report:\n", classification_report(y_train, y_pred_classes))

    print("ALL")
    y_pred = model.predict(X)
    y_pred_classes = (y_pred > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y, y_pred_classes))
    print("Confusion matrix:\n", confusion_matrix(y, y_pred_classes))
    print("Classification report:\n", classification_report(y, y_pred_classes))


TEST
46/46 [==============================] - 0s 3ms/step
Accuracy: 0.6432022084195997
Confusion matrix:
 [[427 299]
 [218 505]]
Classification report:
               precision    recall  f1-score   support

           0       0.66      0.59      0.62       726
           1       0.63      0.70      0.66       723

    accuracy                           0.64      1449
   macro avg       0.65      0.64      0.64      1449
weighted avg       0.65      0.64      0.64      1449

TRAIN
182/182 [==============================] - 0s 2ms/step
Accuracy: 0.7279475228724323
Confusion matrix:
 [[1983  912]
 [ 664 2234]]
Classification report:
               precision    recall  f1-score   support

           0       0.75      0.68      0.72      2895
           1       0.71      0.77      0.74      2898

    accuracy                           0.73      5793
   macro avg       0.73      0.73      0.73      5793
weighted avg       0.73      0.73      0.73      5793

ALL
227/227 [====================